In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
# import googlemaps
# gmaps = googlemaps.Client(key=keys.gmaps_key)
# import pgeocode as pg

from geopy.geocoders import Nominatim, GoogleV3
from geopy.extra.rate_limiter import RateLimiter

import openrouteservice as ors
ors_client = ors.Client(key="5b3ce3597851110001cf6248df561ad4f30343faa8615b35dc108efe")

import osmnx as ox
import networkx as nx
import h3

# get geolocation from address name
## geocode / nominatim

In [2]:
geolocator = Nominatim(user_agent="xxx")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

test = {'street':'27 Hedemannstraße', 'postalcode':'10963', 'city':'Berlin'}
geocode(test, country_codes='de')

Location(27, Hedemannstraße, Kreuzberg, Friedrichshain-Kreuzberg, Berlin, 10963, Deutschland, (52.5029479, 13.3874993, 0.0))

In [3]:
# test_batch = clean_df.apply(lambda x: x['Pick_Up_Housenr']+' '+x['Pick_Up_Street'] \
#                                         +", "+ str(x['Pick_Up_Zipcode'])+', '+'Berlin' , axis=1).drop_duplicates().values

## gmaps

In [ ]:
# gmaps_locator = GoogleV3(api_key=keys.gmaps_key)
# gmaps_geocode = RateLimiter(gmaps_locator.geocode, min_delay_seconds=1)
# gmaps_geocode(query=test_batch[0])

# openrouteservice code

In [ ]:
def get_route_ors(
    route, ors_client=ors_client, return_to_depot=False, profile="driving-car"
):
    """ "cycling-electric" / driving-car"""
    if return_to_depot:
        route = np.concatenate([route, [route[0]]])

    coords = tuple(map(tuple, np.flip(route)))
    geo_json = ors_client.directions(coords, format="geojson", profile=profile)
    return geo_json

def ors_directions(coords, ors_client=ors_client, profile="driving-car"):
    """ "cycling-electric" / driving-car"""
    return ors_client.directions(coords, format="geojson", profile=profile)


## OSMNx code

In [ ]:
def osmnx_route_dist(init_coords, dest_coords, G):
    # get nearest node
    start_node = ox.distance.nearest_nodes(G, *np.flip(init_coords))
    finish_node = ox.distance.nearest_nodes(G, *np.flip(dest_coords))
    route = nx.shortest_path_length(G, start_node, finish_node, weight="length")
    return route


def osmnx_all_route_dist(task_coords_arr, G):
    tot_dist = sum(
        [
            get_route_dist(ori, dest)
            for ori, dest in zip(task_coords_arr[:-1], task_coords_arr[1:])
        ]
    )
    return tot_dist

### plotting

In [ ]:
location_point = tuple(pickup_df[['pu_lat','pu_long']].drop_duplicates().mean(axis=0).values)
Gbike = ox.graph_from_point(location_point, dist=13000, network_type='bike', retain_all=True)
Gdrive = ox.graph_from_point(location_point, dist=20000, network_type='drive', retain_all=True)

In [ ]:
fig1, ax1 = ox.plot_graph(Gbike, figsize=(36,36), bgcolor='#ffffff',
                        show=False, close=False, 
                        edge_color=(220/255,220/255,220/255), node_color='none')

# other 

In [ ]:
def haversine_np(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km